In [1]:
# from pyiceberg.catalog import load_catalog
# from iceberg_table_schema import SCHEMAS
# from pyiceberg.schema import Schema
# from pyiceberg.types import NestedField, IntegerType, StringType, FloatType
# import pyarrow as pa

# schema = Schema(
#     NestedField(field_id=1, name="id", field_type=IntegerType(), required=True),
#     NestedField(field_id=2, name="category", field_type=StringType(), required=True),
#     NestedField(field_id=3, name="amount", field_type=FloatType(), required=True),
# )

# catalog = load_catalog(
#     "default",  # Default catalog name
# )
# catalog.create_namespace_if_not_exists("hello")
# namespaces = catalog.list_namespaces()
# print("Namespaces:", namespaces)
# table_name = "Drivers"
# # schema = SCHEMAS["sales"][table_name] if table_name in SCHEMAS["sales"] else "sales"
# # iceberg_table = catalog.create_table(f"hello.{table_name}", schema=schema)

# iceberg_table = catalog.create_table_if_not_exists(
#     identifier="hello.sales_data", schema=schema
# )

# pa_table_data = pa.Table.from_pylist(
#     [
#         {"id": 1, "category": "electronics", "amount": 299.99},
#         {"id": 2, "category": "clothing", "amount": 79.99},
#         {"id": 3, "category": "groceries", "amount": 45.50},
#         {"id": 4, "category": "electronics", "amount": 999.99},
#         {"id": 5, "category": "clothing", "amount": 120.00},
#     ],
#     schema=iceberg_table.schema().as_arrow(),
# )

# print("Iceberg Table Schema:", iceberg_table.schema)

# iceberg_table.append(df=pa_table_data)
# print(iceberg_table.scan().to_arrow().to_string(preview_cols=10))


# iceberg_table = catalog.load_table(f"sales.sales_data")

# df = iceberg_table.scan(
#     row_filter="status == 'Incomplete' and app_date >= '04/01/2025'",
#     limit=40,
#     selected_fields=("app_no", "type", "status", "app_date"),
# ).to_pandas()


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType

import os
import sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
spark = (
    SparkSession.builder.appName("IcebergLocalDevelopment")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.5.2,org.xerial:sqlite-jdbc:3.42.0.0",
    )
    .config("spark.sql.crossJoin.enabled", "true")
    .config("spark.sql.iceberg.handle-timestamp-without-timezone", "true")
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    )
    .config("spark.sql.catalog.default", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.default.type", "jdbc")
    # .config(
    #     "spark.sql.catalog.default.catalog-impl",
    #     "org.apache.iceberg.jdbc.JdbcCatalog",
    # )
    .config(
        "spark.sql.catalog.default.uri",
        "jdbc:sqlite:/tmp/warehouse/pyiceberg_catalog.db",
    )
    .config(
        "spark.sql.catalog.default.warehouse",
        "hdfs://localhost:9000/user/hive/warehouse",
    )
    .config("spark.sql.catalog.default.jdbc.driver", "org.sqlite.JDBC")
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

25/06/10 12:26:22 WARN Utils: Your hostname, rohitkarki resolves to a loopback address: 127.0.1.1; using 10.13.164.166 instead (on interface wlp4s0)
25/06/10 12:26:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/rohitkarki/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/rohitkarki/.ivy2/cache
The jars for the packages stored in: /home/rohitkarki/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.xerial#sqlite-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ed92da2c-9299-4294-b7e3-74b3f8699de9;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.5.2 in central
	found org.xerial#sqlite-jdbc;3.42.0.0 in central
:: resolution report :: resolve 85ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.5.2 from central in [default]
	org.xerial#sqlite-jdbc;3.42.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      def

25/06/10 12:26:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:

# Check current catalog and namespace
spark.sql("SHOW CURRENT NAMESPACE").show()

# List all namespaces/databases
spark.sql("SHOW NAMESPACES").show()

# List namespaces in the default catalog specifically
spark.sql("SHOW NAMESPACES IN default").show()

# If sales namespace exists, try to use it
spark.sql("USE default.sales")
spark.sql("SHOW TABLES").show()


+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+

+---------+
|namespace|
+---------+
|  default|
+---------+

25/06/10 12:26:29 WARN JdbcCatalog: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1
+---------+
|namespace|
+---------+
|    sales|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|    sales|  Drivers|      false|
|    sales| vehicles|      false|
+---------+---------+-----------+



In [7]:
spark.sql(
    """
SELECT COUNT(*) as count, table.status
FROM sales.Drivers as table
GROUP BY status
ORDER BY COUNT(*) DESC
"""
).show(10)

+-----+--------------------+
|count|              status|
+-----+--------------------+
| 2579|          Incomplete|
|  440|Approved - Licens...|
|  153|              Denied|
|   83|        Under Review|
|    7|Pending Fitness I...|
+-----+--------------------+



In [ ]:
# Set default catalog to local for convenience
# spark.sql("USE default")

# # Verify Spark session creation and catalog
# print("Available catalogs:")
# spark.sql("SHOW CATALOGS").show()

# # # Create namespace/database
# # spark.sql("CREATE NAMESPACE IF NOT EXISTS hello")


# # # Show available namespaces
# # print("Available namespaces:")
# # spark.sql("SHOW NAMESPACES").show()


# # Create sample data
# data = [
#     (1, "electronics", 299.99),
#     (2, "clothing", 79.99),
#     (3, "groceries", 45.50),
#     (4, "electronics", 999.99),
#     (5, "clothing", 120.00),
# ]

# # Define schema
# schema = StructType(
#     [
#         StructField("id", LongType(), True),
#         StructField("category", StringType(), True),
#         StructField("amount", DoubleType(), True),
#     ]
# )

# # # Create DataFrame
# # df = spark.createDataFrame(data, schema)

# # # Write DataFrame to Iceberg table
# # df.write.format("iceberg").mode("overwrite").saveAsTable("hello.sales_data_df")

# # Read the data back
# print("Data written via DataFrame API:")
# spark.table("hello.sales_data_df").show()

# # Perform analytics
# print("Category-wise totals:")
# spark.sql(
#     """
# SELECT category, 
#        COUNT(*) as count, 
#        SUM(amount) as total_amount,
#        AVG(amount) as avg_amount
# FROM hello.sales_data_df 
# GROUP BY category 
# ORDER BY total_amount DESC
# """
# ).show()


25/06/10 10:51:58 WARN JdbcCatalog: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1
Available catalogs:
+-------+
|catalog|
+-------+
|default|
+-------+

Data written via DataFrame API:


+---+-----------+------+
| id|   category|amount|
+---+-----------+------+
|  1|electronics|299.99|
|  6|      books| 25.99|
|  2|   clothing| 79.99|
|  3|  groceries|  45.5|
|  7|electronics|599.99|
|  4|electronics|999.99|
|  5|   clothing| 120.0|
+---+-----------+------+

Category-wise totals:
+-----------+-----+------------+-----------------+
|   category|count|total_amount|       avg_amount|
+-----------+-----+------------+-----------------+
|electronics|    3|     1899.97|633.3233333333334|
|   clothing|    2|      199.99|           99.995|
|  groceries|    1|        45.5|             45.5|
|      books|    1|       25.99|            25.99|
+-----------+-----+------------+-----------------+

